In [1]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 45.1 MB/s eta 0:00:00


In [3]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re

In [9]:
# Load and preprocess data
df = pd.read_csv('merged_output.csv')

# Load positive and depressed words
with open('positive-words.txt', 'r') as file:
    positive_words = set(file.read().splitlines())

with open('depressedword.txt', 'r') as file:
    depressed_words = set(file.read().splitlines())

# Function to label the sentiment
def label_sentiment(text):
    words = set(text.lower().split())
    positive_count = len(words.intersection(positive_words))
    depressed_count = len(words.intersection(depressed_words))
    
    if depressed_count > positive_count:
        return 2  # Depressed
    elif positive_count > 0:
        return 1  # Positive
    else:
        return 0  # Neutral/Negative

# Create labels based on word counts
df['label'] = df['content'].apply(label_sentiment)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['label'], test_size=0.2, random_state=42)

# Tokenize the text
max_words = 670000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [11]:
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Create TF-IDF features
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train_encoded)

# Make predictions
rf_predictions = rf_model.predict(X_test_tfidf)

# Evaluate Random Forest model
print("Random Forest Classifier Results:")
print("Accuracy:", accuracy_score(y_test_encoded, rf_predictions))
print("Classification Report:")
print(classification_report(y_test_encoded, rf_predictions))

# XGBoost Classifier
xgb_model = XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train_tfidf, y_train_encoded)

# Make predictions
xgb_predictions = xgb_model.predict(X_test_tfidf)

# Evaluate XGBoost model
print("\nXGBoost Classifier Results:")
print("Accuracy:", accuracy_score(y_test_encoded, xgb_predictions))
print("Classification Report:")
print(classification_report(y_test_encoded, xgb_predictions))

NameError: name 'le' is not defined

In [17]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load and preprocess data
df = pd.read_csv('merged_output.csv')

# Load positive and depressed words
with open('positive-words.txt', 'r') as file:
    positive_words = set(file.read().splitlines())

with open('depressedword.txt', 'r') as file:
    depressed_words = set(file.read().splitlines())

# Function to label the sentiment
def label_sentiment(text):
    words = set(text.lower().split())
    positive_count = len(words.intersection(positive_words))
    depressed_count = len(words.intersection(depressed_words))
    
    if depressed_count > positive_count:
        return 2  # Depressed
    elif positive_count > 0:
        return 1  # Positive
    else:
        return 0  # Neutral/Negative

# Create labels based on word counts
df['label'] = df['content'].apply(label_sentiment)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['label'], test_size=0.2, random_state=42)

# Vectorization using TF-IDF
tfidf = TfidfVectorizer(max_features=670000, stop_words='english')
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Encode labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)




In [19]:
# Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train_encoded)

# Predictions
rf_predictions = rf_model.predict(X_test_tfidf)

# Evaluate Random Forest
print("Random Forest Classifier Report:")
print(classification_report(y_test_encoded, rf_predictions))
print(f"Accuracy: {accuracy_score(y_test_encoded, rf_predictions)}")

Random Forest Classifier Report:
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      2032
           1       0.95      0.81      0.88      1026
           2       0.93      0.52      0.67       362

    accuracy                           0.89      3420
   macro avg       0.92      0.78      0.82      3420
weighted avg       0.90      0.89      0.88      3420

Accuracy: 0.8891812865497076


In [15]:

# XGBoost Classifier
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_model.fit(X_train_tfidf, y_train_encoded)

# Predictions
xgb_predictions = xgb_model.predict(X_test_tfidf)

# Evaluate XGBoost
print("XGBoost Classifier Report:")
print(classification_report(y_test_encoded, xgb_predictions))
print(f"Accuracy: {accuracy_score(y_test_encoded, xgb_predictions)}")

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:57:18] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Classifier Report:
              precision    recall  f1-score   support

           0       0.85      1.00      0.92      2032
           1       0.97      0.78      0.86      1026
           2       0.91      0.51      0.65       362

    accuracy                           0.88      3420
   macro avg       0.91      0.76      0.81      3420
weighted avg       0.89      0.88      0.87      3420

Accuracy: 0.8809941520467837


In [21]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load and preprocess data
df = pd.read_csv('merged_output.csv')

# Load positive and depressed words
with open('positive-words.txt', 'r') as file:
    positive_words = set(file.read().splitlines())

with open('depressedword.txt', 'r') as file:
    depressed_words = set(file.read().splitlines())

# Function to label the sentiment
def label_sentiment(text):
    words = set(text.lower().split())
    positive_count = len(words.intersection(positive_words))
    depressed_count = len(words.intersection(depressed_words))
    
    if depressed_count > positive_count:
        return 2  # Depressed
    elif positive_count > 0:
        return 1  # Positive
    else:
        return 0  # Neutral/Negative

# Create labels based on word counts
df['label'] = df['content'].apply(label_sentiment)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['label'], test_size=0.2, random_state=42)

# Vectorization using TF-IDF
tfidf = TfidfVectorizer(max_features=670000, stop_words='english')
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Encode labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Define the classifiers
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Create a Voting Classifier
voting_model = VotingClassifier(estimators=[
    ('rf', rf_model),
    ('xgb', xgb_model)
], voting='soft')  # 'soft' voting uses predicted probabilities

# Fit the Voting Classifier
voting_model.fit(X_train_tfidf, y_train_encoded)

# Predictions from the Voting Classifier
voting_predictions = voting_model.predict(X_test_tfidf)

# Evaluate the Voting Classifier
print("Voting Classifier Report:")
print(classification_report(y_test_encoded, voting_predictions))
print(f"Accuracy: {accuracy_score(y_test_encoded, voting_predictions)}")

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:59:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Voting Classifier Report:
              precision    recall  f1-score   support

           0       0.86      1.00      0.92      2032
           1       0.97      0.80      0.88      1026
           2       0.92      0.53      0.67       362

    accuracy                           0.89      3420
   macro avg       0.92      0.78      0.82      3420
weighted avg       0.90      0.89      0.88      3420

Accuracy: 0.8900584795321638


In [23]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Define base models
base_models = [
    ('rf', rf_model),
    ('xgb', xgb_model)
]

# Define the meta model (Logistic Regression in this case)
meta_model = LogisticRegression()

# Create a Stacking Classifier
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model)

# Fit the Stacking Classifier
stacking_model.fit(X_train_tfidf, y_train_encoded)

# Predictions from the Stacking Classifier
stacking_predictions = stacking_model.predict(X_test_tfidf)

# Evaluate the Stacking Classifier
print("Stacking Classifier Report:")
print(classification_report(y_test_encoded, stacking_predictions))
print(f"Accuracy: {accuracy_score(y_test_encoded, stacking_predictions)}")

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:59:50] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:00:25] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:00:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:00:27] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packa

Stacking Classifier Report:
              precision    recall  f1-score   support

           0       0.89      0.99      0.93      2032
           1       0.94      0.83      0.88      1026
           2       0.90      0.65      0.75       362

    accuracy                           0.90      3420
   macro avg       0.91      0.82      0.86      3420
weighted avg       0.91      0.90      0.90      3420

Accuracy: 0.9038011695906433
